# mastodon API

read data of accounts you are following and store it in excel file.

+ utilizes `Mastodon.py` package to read account information
+ `pandas` is used to create excel file
  + sheet for accounts following
  + sheet for status per day, find loud and mute accounts
+ you may want to remove inactive accounts, which haven't tooted at all, set `iLimitMute` to non negative value
+ you may want to remove loud accounts, those tooting too much per day, set `iLimitLoud` to a positive number



background links, references

+ [Mastodon.py](https://pypi.org/project/Mastodon.py/)
  + [pypi](https://pypi.org/project/Mastodon.py/)
  + [doc](https://mastodonpy.readthedocs.io/en/stable/)
  + [repo](https://github.com/halcy/Mastodon.py)
+ [mastodon](https://joinmastodon.org/)
  + [API getting started](https://docs.joinmastodon.org/client/intro/)
  + [pagination](https://docs.joinmastodon.org/api/guidelines/#pagination)

**setup**

```shell
pip install -r requirements.txt
```

In [ ]:
sSecretFN='.env.py.secret'
sOutFN='me_following.xlsx'
sSht='following'
sShtDays='status_per_day'
iLimitMute=0                # inactive accounts
iLimitLoud=120              # accounts above this threshold are unfollowed

create an access token through profile, go to `</> Development`

+ `New Application`
  + provide meaningful application name
  + define scopes, grant permission for application
  + `submit`
+ click on application name
  + copy value of `your access token`
  + store it to file `sSecretFN`
  + `save changes`

minimal content of `sSecretFN`

```
access_token="<your access token>"
api_base_url="https://<instance name>"
```

do not expose your tokens to the repo, add line to `.gitignore`

```
**/.env.*
```


create an access token through python, do this only once

```python
from mastodon import Mastodon

oMstdn = Mastodon(
    client_id = '<your app>',
    api_base_url = 'https://<your instance>'
)
oMstdn.log_in(
    'youremail',
    'superseretpasswork',
    to_file = '.env.py.secret'
)
```

**THE MIT license (MIT)**

Copyright © 2022 Walter Obweger (WRO)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
import os
import time
import datetime
import pprint
import traceback
import random
import pandas

from dotenv import load_dotenv
from mastodon import Mastodon


In [ ]:
iVerbose=20

In [ ]:
load_dotenv(sSecretFN)
print(os.environ.get('api_base_url'))

In [ ]:
oMstdn = Mastodon(
    access_token = os.environ.get('access_token'),
    api_base_url = os.environ.get('api_base_url')
)

In [ ]:
sVer=oMstdn.retrieve_mastodon_version()
print(sVer)

get me for mastodon instance

In [ ]:
dMe=oMstdn.me()

In [ ]:
pprint.pp(dMe)

In [ ]:
print(dMe.id)

In [ ]:
dMe.created_at.strftime('%Y-%m-%d')

In [ ]:
print(dMe.url)

get first page of me following

In [ ]:
lFollowing=oMstdn.account_following(dMe.id)

get keys from first element 

In [ ]:
print(list(lFollowing[0].keys()))

In [ ]:
def addAccountToDict(lFollowing,lFollow):
    """iterate over following list page and store relevant account data in list
    """
    for dAcc in lFollowing:
        dA={
            'id':dAcc.id,
            'username':dAcc.username,
            'display_name':dAcc.display_name,
            'acct':dAcc.acct,
            'created_at':dAcc.created_at.strftime('%Y-%m-%d'),
            'statuses_count':dAcc.statuses_count,
            'followers_count':dAcc.followers_count,
            'following_count':dAcc.following_count,
            'discoverable':dAcc.discoverable,
            'locked':dAcc.locked,
            'group':dAcc.group,
            'bot':dAcc.bot,
            'url':dAcc.url,
            'note':dAcc.note,

        }
        if dAcc.last_status_at is not None:
            dAcc['last_status_at']=dAcc.last_status_at.strftime('%Y-%m-%d')
        lFollow.append(dA)

convert following account data to list and use pagination to iterate through all accounts

In [ ]:
lFollow=[]
try:
    while (lFollowing is not None):
        addAccountToDict(lFollowing,lFollow)
        lFollowingNext=oMstdn.fetch_next(lFollowing)
        if iVerbose>20:
            print(len(lFollowingNext))
        lFollowing=lFollowingNext
        time.sleep(0.1+random.random()*0.25)
except:
    print(traceback.format_exc())

if iVerbose>0:
    print('following count:%d'%(len(lFollow)))
if iVerbose>20:
    pprint.pp(lFollow[:5])

In [ ]:
print('following count:%d'%(len(lFollow)))

In [ ]:
def applyFmt(oExcelWr,sSht,lCol):
    oBok=oExcelWr.book
    oSht=oExcelWr.sheets[sSht]
    oFmt=oBok.add_format({
                        'text_wrap':False,
                        'align':'left',
                        'valign':'top',
                        })
    for sAdr,iWidth in lCol:
        oSht.set_column(sAdr,iWidth,oFmt)

In [ ]:
lCol=[
    ('A:A',25),
    ('B:C',20),
    ('D:D',40),
    ('E:E',15),
    ('F:H',8),
    ('I:L',8),
    ('M:M',15),
    ('N:N',15),
]

In [ ]:
lColDays=[
    ('A:A',25),
    ('B:B',50),
    ('C:C',15),
    ('D:D',10),
    ('E:F',15),

]

In [ ]:
dfFollowing=pandas.DataFrame(data=lFollow,index=None,
        dtype='string')

In [ ]:
dfFollowing['statuses_count']=pandas.to_numeric(dfFollowing['statuses_count'])
dfFollowing['followers_count']=pandas.to_numeric(dfFollowing['followers_count'])
dfFollowing['following_count']=pandas.to_numeric(dfFollowing['following_count'])

In [ ]:
zToDay=datetime.date.today()

In [ ]:
dfFollowing.head()

In [ ]:
def calcDys(x):
    try:
        zCreate=datetime.date.fromisoformat(x['created_at'])
        oDiff=zToDay-zCreate
        iDy=oDiff.days
        x['dys']=iDy
        x['status_per_day']=x['statuses_count']/iDy
        return x
    except:
        x['dys']=-1
        return x

In [ ]:
dfDaysTmp=dfFollowing.apply(calcDys,axis=1)

In [ ]:
dfDaysTmp.sort_values(['status_per_day'],inplace=True,ascending=False)

In [ ]:
dfDays=dfDaysTmp[['id','acct','status_per_day','dys','statuses_count','group']]

In [ ]:
dfDays.head()

In [ ]:
dfFollowing.sort_values(['username'],inplace=True)

write following information

In [ ]:
with pandas.ExcelWriter(sOutFN,mode='w') as oXlsWr:
    dfFollowing.to_excel(oXlsWr,sheet_name=sSht,index=False)
    applyFmt(oXlsWr,sSht,lCol)
    if sShtDays is not None:
        dfDays.to_excel(oXlsWr,sheet_name=sShtDays,index=False)
        applyFmt(oXlsWr,sShtDays,lColDays)

find mute accounts, those not tooting

In [ ]:
dfMute=dfFollowing[dfFollowing['statuses_count']<=iLimitMute]

In [ ]:
print(dfMute.shape)

unfollow account not tooting

In [ ]:
if iLimitMute>=0:
    for iOfs,oRow in dfMute.iterrows():
        print(iOfs,oRow['id'])
        oMstdn.account_unfollow(oRow['id'])

find individuals

In [ ]:
dfIndividuals=dfDays[dfDays['group']=='False']

find too loud individuals

In [ ]:
dfLoud=dfIndividuals[dfIndividuals['status_per_day']>iLimitLoud]

In [ ]:
dfLoud.shape

unfollowing too loud individuals

In [ ]:
if iLimitLoud>0:
    for iOfs,oRow in dfLoud.iterrows():
        print(iOfs,oRow['id'])
        oMstdn.account_unfollow(oRow['id'])